In [1]:
# dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as stats
import numpy as np
from scipy.stats import linregress
import requests
import time
import json

master_csv = "archive/Master.csv"
salaries_csv = "archive/Salaries.csv"
teams_csv = "archive/Teams.csv"

In [2]:
#read in csv files
master_df = pd.read_csv(master_csv)
#master_df.head()
salaries_df = pd.read_csv(salaries_csv)
#salaries_df.head()
teams_df = pd.read_csv(teams_csv)
#teams_df

#filter batting df 
#filtered_batting_df = batting_df.loc[batting_df['yearID'] > 1999]
#filtered_batting_df

# Combine the data into a single dataset
bball_data_merge = pd.merge(master_df, salaries_df, how="left", on="playerID")

# Display the data table for preview
bball_data_merge.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,bats,throws,debut,finalGame,retroID,bbrefID,yearID,teamID,lgID,salary
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,2004.0,SFN,NL,300000.0
1,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,2007.0,CHA,AL,387500.0
2,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,2008.0,BOS,AL,403250.0
3,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,2009.0,SEA,AL,419000.0
4,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,2010.0,SEA,AL,2750000.0


In [3]:
#list(bball_data_merge.columns) 

In [4]:
#drop unneed columns
cleaned_df = bball_data_merge.drop(['birthMonth',
                                    'birthDay',
                                    'deathYear',
                                    'deathMonth',
                                    'deathDay',
                                    'deathCountry',
                                    'deathState',
                                    'deathCity',
                                    'nameFirst',
                                    'nameLast',
                                    'nameGiven',
                                    'weight',
                                    'height',
                                    'bats',
                                    'throws',
                                    'debut',
                                    'finalGame',
                                    'retroID',
                                    'bbrefID',                                   
                                     ], axis = 1)

cleaned_df.head()

,playerID,birthYear,birthCountry,birthState,birthCity,yearID,teamID,lgID,salary
0,aardsda01,1981.0,USA,CO,Denver,2004.0,SFN,NL,300000.0
1,aardsda01,1981.0,USA,CO,Denver,2007.0,CHA,AL,387500.0
2,aardsda01,1981.0,USA,CO,Denver,2008.0,BOS,AL,403250.0
3,aardsda01,1981.0,USA,CO,Denver,2009.0,SEA,AL,419000.0
4,aardsda01,1981.0,USA,CO,Denver,2010.0,SEA,AL,2750000.0


In [5]:
#list(cleaned_df.columns) 

In [6]:
#df.dropna(subset = ["column2"], inplace=True)

#Rename cleaned_df columns and change df to salary_df
salary_df = cleaned_df.rename(columns={'playerID': 'Player ID',
                                         'birthYear': 'Birth Year',
                                         'birthCountry': 'Birth Country',
                                         'birthState': 'Birth State',
                                         'birthCity': 'Birth City',
                                         'yearID': 'Year ID',
                                         'teamID': 'Team ID',
                                         'lgID': 'League',
                                         'salary':'Salary'
                                     })

#Narrow down dataset to 2000 - 2015 years and remove NaN values in Salary column
salary_df = salary_df.loc[salary_df['Salary'].isnull() == False]
salary_df = salary_df.loc[salary_df['Year ID'] > 1999]
salary_df = pd.DataFrame(salary_df)
salary_df.head()

,Player ID,Birth Year,Birth Country,Birth State,Birth City,Year ID,Team ID,League,Salary
0,aardsda01,1981.0,USA,CO,Denver,2004.0,SFN,NL,300000.0
1,aardsda01,1981.0,USA,CO,Denver,2007.0,CHA,AL,387500.0
2,aardsda01,1981.0,USA,CO,Denver,2008.0,BOS,AL,403250.0
3,aardsda01,1981.0,USA,CO,Denver,2009.0,SEA,AL,419000.0
4,aardsda01,1981.0,USA,CO,Denver,2010.0,SEA,AL,2750000.0


In [ ]:
usa_salary_df = salary_df.loc[salary_df['Birth Country'] == 'USA']
dr_salary_df = salary_df.loc[salary_df['Birth Country'] == 'D.R.']
ven_salary_df = salary_df.loc[salary_df['Birth Country'] == 'Venezuela']
pr_salary_df = salary_df.loc[salary_df['Birth Country'] == 'P.R.']
can_salary_df = salary_df.loc[salary_df['Birth Country'] == 'CAN']
jap_salary_df = salary_df.loc[salary_df['Birth Country'] == 'Japan']

#line plot
plt.scatter(usa_salary_df['Year ID'], usa_salary_df['Salary'], color = "red", label='USA')
plt.scatter(dr_salary_df['Year ID'], dr_salary_df['Salary'], color = "blue", label='D.R.')
#plt.plot(ven_salary_df['Year ID'], ven_salary_df['Salary'], color = "green", label='Venezuela')
#plt.plot(pr_salary_df['Year ID'], pr_salary_df['Salary'], color = "blue", label='P.R.')
plt.title("")
plt.xlabel("", color = "orange")
plt.xticks(rotation = 90)
plt.ylabel("", color = "green")
plt.show()

In [ ]:
#group by birth city
birth_city = salary_df.groupby(['Birth City'])
cities = []
cities.append(salary_df['Birth City'])


#create each variable
player_count = birth_city['Player ID'].nunique()
mean_salary = birth_city.mean()['Salary']
median_salary = birth_city.median()['Salary']
var_salary = birth_city.var()['Salary']
std_dev_salary = birth_city.std()['Salary']
sems_salary = birth_city.sem()['Salary']

#player birth city summary table
summary_city_df = pd.DataFrame({"Number of Players": player_count,
                              "Mean Player Salary": mean_salary,
                              "Median Player Salary": median_salary,
                              "Player Salary Variance": var_salary,
                              "Player Salary Std. Dev.": std_dev_salary,
                              "Player Salary Std. Error": sems_salary})

#format columns
summary_city_df = summary_city_df.sort_values(by=['Number of Players'], ascending=False)
summary_city_df['Mean Player Salary'] = summary_city_df['Mean Player Salary'].map("${:,.0f}".format)
summary_city_df['Median Player Salary'] = summary_city_df['Median Player Salary'].map("${:,.0f}".format)
summary_city_df['Player Salary Variance'] = summary_city_df['Player Salary Variance'].map("${:,.0f}".format)
summary_city_df['Player Salary Std. Dev.'] = summary_city_df['Player Salary Std. Dev.'].map("${:,.0f}".format)
summary_city_df['Player Salary Std. Error'] = summary_city_df['Player Salary Std. Error'].map("${:,.0f}".format)

output_summary_city = summary_city_df.reset_index()
summary_city_df

In [ ]:
# Save summary_city_df as a csv
# Note to avoid any issues later, use encoding="utf-8"
output_summary_city.to_csv("salary_summary_city_WJR.csv", encoding="utf-8", index=False)

In [ ]:
#group by birth country
birth_country = salary_df.groupby(['Birth Country'])

#create each variable
player_count = birth_country['Player ID'].nunique()
mean_salary = birth_country.mean()['Salary']
median_salary = birth_country.median()['Salary']
max_salary = birth_country['Salary'].max()
min_salary = birth_country['Salary'].min()
var_salary = birth_country.var()['Salary']
std_dev_salary = birth_country.std()['Salary']
sems_salary = birth_country.sem()['Salary']

#player birth city summary table
summary_country_df = pd.DataFrame({"Number of Players": player_count,
                              "Mean Player Salary": mean_salary,
                              "Median Player Salary": median_salary,
                               "Max Player Salary": max_salary,
                                "Min Player Salary": min_salary,
                              "Player Salary Variance": var_salary,
                              "Player Salary Std. Dev.": std_dev_salary,
                              "Player Salary Std. Error": sems_salary})

#format columns
summary_country_df = summary_country_df.sort_values(by=['Number of Players'], ascending=False)
summary_country_df['Mean Player Salary'] = summary_country_df['Mean Player Salary'].map("${:,.0f}".format)
summary_country_df['Median Player Salary'] = summary_country_df['Median Player Salary'].map("${:,.0f}".format)
summary_country_df['Max Player Salary'] = summary_country_df['Max Player Salary'].map("${:,.0f}".format)
summary_country_df['Min Player Salary'] = summary_country_df['Min Player Salary'].map("${:,.0f}".format)
summary_country_df['Player Salary Variance'] = summary_country_df['Player Salary Variance'].map("${:,.0f}".format)
summary_country_df['Player Salary Std. Dev.'] = summary_country_df['Player Salary Std. Dev.'].map("${:,.0f}".format)
summary_country_df['Player Salary Std. Error'] = summary_country_df['Player Salary Std. Error'].map("${:,.0f}".format)

summary_country_df

In [ ]:
usa_salary_df = salary_df.loc[salary_df['Birth Country'] == 'USA']
usa_salary_df

In [ ]:
#line plot
plt.bar(usa_salary_df['Year ID'], usa_salary_df['Salary'], color = "purple")
plt.title("Avg. Salary vs. State")
plt.xlabel("State", color = "orange")
plt.xticks(rotation = 90)
plt.ylabel("Salary", color = "green")
plt.show()

In [ ]:
#group by birth state
birth_state = usa_salary_df.groupby(['Birth State'])

#print(birth_state)
#create each variable
player_count = birth_state['Player ID'].nunique()
mean_salary = birth_state.mean()['Salary']
median_salary = birth_state.median()['Salary']
max_salary = birth_state['Salary'].max()
min_salary = birth_state['Salary'].min()
var_salary = birth_state.var()['Salary']
std_dev_salary = birth_state.std()['Salary']
sems_salary = birth_state.sem()['Salary']

#player birth city summary table
summary_state_df = pd.DataFrame({"Number of Players": player_count,
                              "Mean Player Salary": mean_salary,
                              "Median Player Salary": median_salary,
                              "Player Salary Variance": var_salary,
                              "Player Salary Std. Dev.": std_dev_salary,
                              "Player Salary Std. Error": sems_salary})

#format columns
summary_state_df = summary_state_df.sort_values(by=['Mean Player Salary'], ascending=False)
summary_state_df['Mean Player Salary'] = summary_state_df['Mean Player Salary'].map("${:,.0f}".format)
summary_state_df['Median Player Salary'] = summary_state_df['Median Player Salary'].map("${:,.0f}".format)
summary_state_df['Player Salary Variance'] = summary_state_df['Player Salary Variance'].map("${:,.0f}".format)
summary_state_df['Player Salary Std. Dev.'] = summary_state_df['Player Salary Std. Dev.'].map("${:,.0f}".format)
summary_state_df['Player Salary Std. Error'] = summary_state_df['Player Salary Std. Error'].map("${:,.0f}".format)

summary_state_df.reset_index()

In [ ]:
#line plot
plt.bar(summary_state_df['Birth State'], summary_state_df['Mean Player Salary'], color = "purple")
plt.title("Avg. Salary vs. State")
plt.xlabel("State", color = "orange")
plt.xticks(rotation = 90)
plt.ylabel("Salary", color = "green")
plt.show()

In [ ]:
#MLB city salary summary
#city salary summary data file
city_salary_path = "../group_1_project/salary_summary_city_WJR.csv"

#read the city salary data file
city_salary_df = pd.read_csv(city_salary_path, dtype="object", encoding="utf-8")
city_salary_df.head() 